In [4]:
# %pip install langchain openai langchain_community langchain_openai huggingface_hub
# !pip install transformers
# !pip install sentence_transformers
# !pip install gradio
# !pip install natasha
# !pip install nltk

In [5]:
# импорты
import gradio as gr
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer,SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from natasha import (Segmenter,MorphVocab,NewsEmbedding,
    NewsMorphTagger,NewsSyntaxParser,NewsNERTagger,
    PER,NamesExtractor,Doc)
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm


In [6]:
# модели llm
# для промтов
# base_url = "http://hackllm.vkcloud.eazify.net:8000/v1" # Mail.ru
# base_url = "http://saiga.vkcloud.eazify.net:8000/v1" # Saiga 3
promt_model = ChatOpenAI(api_key="<key>",
                         model = "tgi",
                         openai_api_base="http://saiga.vkcloud.eazify.net:8000/v1")

summarize_model = T5ForConditionalGeneration.from_pretrained('cointegrated/rut5-base-absum')
summarize_tokenizer = T5Tokenizer.from_pretrained('cointegrated/rut5-base-absum')
summarize_model.eval()
# NER
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
# for validate result of promt model
get_tag_model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

# функции pipeline для определения тэгов
def summarize(text, n_words=None, compression=None,max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0,
    **kwargs):
    '''Функция генерации обобщения новости при помощи русской t5'''

    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = summarize_tokenizer(text, return_tensors='pt', padding=True).to(summarize_model.device)
    with torch.inference_mode():
        out = summarize_model.generate(
            **x,
            max_length=max_length, num_beams=num_beams,
            do_sample=do_sample, repetition_penalty=repetition_penalty,
            **kwargs
        )
    return summarize_tokenizer.decode(out[0], skip_special_tokens=True)

def get_tag(llm_ans, tag_lst):
    query_embedding = get_tag_model.encode(llm_ans)
    passage_embedding = get_tag_model.encode(tag_lst)
    dists = util.dot_score(query_embedding, passage_embedding).numpy()[0]
    return tag_lst[dists.argmax()]

def preprocess_text(text):
    stop_words = set(stopwords.words('russian'))
    stemmer = SnowballStemmer("russian")

    words = word_tokenize(text.lower())
    filtered_words = [stemmer.stem(word) for word in words if word.isalnum() and word not in stop_words]

    return ' '.join(filtered_words)

def find_closest_text(user_text, group_texts):
    preprocessed_user_text = preprocess_text(user_text)
    preprocessed_group_texts = [preprocess_text(text) for text in group_texts]

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([preprocessed_user_text] + preprocessed_group_texts)

    similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:])

    closest_index = similarities.argmax()

    return group_texts[closest_index]

def big_tag_news_with_chat_model(chat_model,system_promt,human_promt,
                                 news_text,
                                 news_desc = None,
                                 big_tags = ['Общественные и социальные темы',
                                            'Искусство',
                                            'Красота и Здоровье',
                                            'Наука и техника',
                                            'Спорт',
                                            'Экономика',
                                            'Строительство и Интерьер',
                                            'Развлечение',
                                            'Другое'],
                                get_tag_func = get_tag):
    '''Функция по выявлению тэга первой абстракции на основе краткого описания новости'''

    if news_desc is None:
        news_desc = summarize(news_text)

    if '....' in human_promt:
        human_promt = human_promt.replace('....',"', '".join(big_tags))

    doc = Doc(news_text)

    if doc.spans is not None:
        ners = [ner.text for ner in doc.spans]
        human_promt += f". Кстати, я еще выделил некоторые сущности из текста новости: {', '.join(ners)}"

    messages = [
    SystemMessage(content=system_promt.replace('...',"', '".join(big_tags))),
    HumanMessage(content=human_promt.replace('...',news_desc)),
    ]

    res = chat_model.invoke(messages)

    return get_tag_func(res.content, big_tags)


def small_tag_news_with_chat_model(big_tag,chat_model,
                                   system_promt,human_promt,
                                   news_text,
                                   small_tags = {
                                       'Общественные и социальные темы' : ['Политика','Общество','Международные события','Образование','Здравоохранение','История','Закон и право'],
                                       'Искусство' : ['Кино','Телевидение','Музыка','Литература','Персоны'],
                                       'Красота и Здоровье' : ['Рецепты','Мода','Красота','Косметика','Уход за собой','Медицина','Тело'],'Наука и техника' : ['Наука','Гаджеты','Крипта','Технологии','Автомобили'],
                                       'Спорт' : ['Футбол','Баскетбол','Теннис','Хоккей','Легая атлетика','Фигурное катание','Единоборства','Другой спорт'],
                                       'Экономика' : ['Экономика','Финансы','Бренды','Бизнес','Инвестиции'],
                                       'Строительство и Интерьер':['Дом','Дизайн','Ремонт','Сантехника','Архитектура'],
                                       'Развлечение' : ['Социальные сети','Опросы','Головоломки','Туризм']},
                                   get_tag_func = get_tag):

    '''Функция по выявлению тэга второй абстракции на основе полного текста новости'''

    #     заполним сферу экспертизы для системного промта
    system_promt = system_promt.replace('....',big_tag)

    messages = [
    SystemMessage(content=system_promt.replace('...',"', '".join(small_tags[big_tag]))),
    HumanMessage(content=human_promt.replace('...',news_text)),
    ]
    res = chat_model.invoke(messages)

    return get_tag_func(res.content, small_tags[big_tag])

# промты
system_content_big_tag_1 =  '''Ты - редактор новостей. Твоя задача относить новости к определенным группам на основе краткого описания, которое обернуто в кавычки. Основные твои группы: ...'''
system_content_small_tag_1 = '''Ты эксперт в области "....". Твоя задача определить тэг новости из твоей области, текст новоси обернут в кавычки. Ты можешь выбирать из следующих тэгов:  ...'''
human_content_3 = "Привет! Какая из известных тебе групп (напоминаю группы: ....) лучше всего описывает тему данного описания новости: '...'."
human_content_4 = "Привет! К какому из извесных тебе тэгов лучше всего отнести текст новости: '...'."

big_tags = ['Общественные и социальные темы',
            'Искусство',
            'Красота и Здоровье',
            'Наука и техника',
            'Спорт',
            'Экономика',
            'Строительство и Интерьер',
            'Развлечение',
            'Другое']

small_tags = {'Общественные и социальные темы' : ['Политика','Общество','Международные события','Образование','Здравоохранение','История','Закон и право'],
              'Искусство' : ['Кино','Телевидение','Музыка','Литература','Персоны'],
              'Красота и Здоровье' : ['Рецепты','Мода','Красота','Косметика','Уход за собой','Медицина','Тело'],
              'Наука и техника' : ['Наука','Гаджеты','Крипта','Технологии','Автомобили'],
              'Спорт' : ['Футбол','Баскетбол','Теннис','Хоккей','Легая атлетика','Фигурное катание','Единоборства','Другой спорт'],
              'Экономика' : ['Экономика','Финансы','Бренды','Бизнес','Инвестиции'],
              'Строительство и Интерьер':['Дом','Дизайн','Ремонт','Сантехника','Архитектура'],
              'Развлечение' : ['Социальные сети','Опросы','Головоломки','Туризм']}


def news_tagger(news_text):
    # summary of news
    summary = summarize(news_text)
    result_big_tag = big_tag_news_with_chat_model(promt_model,
                                              system_content_big_tag_1,
                                              human_content_3,
                                              news_text,summary)
    result_small_tag = small_tag_news_with_chat_model(result_big_tag,
                                                      promt_model,
                                                      system_content_small_tag_1,
                                                      human_content_4,
                                                      news_text)
    tag = f'Полученные тэги: {result_big_tag} --> {result_small_tag}'

    return summary, tag


def news_big_tagger(news_text):
    # summary of news
    summary = summarize(news_text)
    result_big_tag = big_tag_news_with_chat_model(promt_model,
                                              system_content_big_tag_1,
                                              human_content_3,
                                              news_text,summary)
    return result_big_tag

config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Подгрузка данные

In [7]:
news_other = pd.read_csv('full_df.csv',index_col=0)
news_other['tag'] = news_other.tag.map(lambda x: x if x in ['technologies','entertainment','food','shopping','architecture'] else 'Спорт')
news_other = news_other[news_other.tag.isin(['technologies','entertainment','food','shopping','architecture','Спорт'])]\
                    .replace({"tag": {'technologies' : 'Наука и техника',
                                      'entertainment' : 'Развлечение',
                                      'food' : 'Красота и Здоровье',
                                      'shopping' : 'Красота и Здоровье',
                                      'architecture' : 'Строительство и Интерьер',
                                     'people' : 'Общественные и социальные темы',}})[['text','tag']]

news_other_sample = news_other.groupby('tag').sample(50)
news_mail = []
sheetname2tag = {'kino.mail':'Искусство',
              'health.mail' : 'Красота и Здоровье',
              'deti.mail' : 'Общественные и социальные темы',
              'dom.mail' : 'Строительство и Интерьер'}

for list_name in ['kino.mail','health.mail','deti.mail','dom.mail']:
    news_list = pd.read_excel('data.mail.xlsx',sheet_name=list_name,nrows=1000)
    news_list['tag'] = sheetname2tag[list_name]
    news_mail.append(news_list)

news_mail = pd.concat(news_mail)[['Article Text','tag']].reset_index(drop=True).groupby('tag').sample(100)
news_mail.columns = ['text','tag']

news = pd.concat([news_mail,news_other_sample]).reset_index(drop=True)

# Расчет тэгов и сравнение с базовыми

In [8]:
test_sample = news.groupby('tag').sample(20)

In [9]:
from tqdm.notebook import tqdm

In [17]:
results = []

for idx,news in tqdm(list(test_sample.iterrows())):
    result = {'news' : news['text'],'tag' : news['tag'], 'pred_tag' : news_big_tagger(news['text'])}
    results.append(result)

results = pd.DataFrame(results)

In [12]:
from sklearn.metrics import classification_report,accuracy_score

In [ ]:
# метрики
results = pd.DataFrame(results)
print(classification_report(results['tag'],results['pred_tag']))